<a href="https://colab.research.google.com/github/aarmentamna/machine_learning_advance/blob/main/TC4033_Activity4_42.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Maestría en Inteligencia Artificial Aplicada**
### **Curso: ADVANCED MACHINE LEARNING METHODS**
## Tecnológico de Monterrey
### Dr. José Antonio Cantoral Ceballos

## Activity Week 9
### **Text Generator.**

*TEAM MEMBERS:*

*   Roberto Romero Vielma - A00822314
*   José Javier Granados Hernández - A00556717
*   Aquiles Yonatan Armenta Hernandez - A01793252
*   Alan Avelino Fernández Juárez - A00989308

## TC 5033
### Text Generation

<br>

#### Activity 4: Building a Simple LSTM Text Generator using WikiText-2
<br>

- Objective:
    - Gain a fundamental understanding of Long Short-Term Memory (LSTM) networks.
    - Develop hands-on experience with sequence data processing and text generation in PyTorch. Given the simplicity of the model, amount of data, and computer resources, the text you generate will not replace ChatGPT, and results must likely will not make a lot of sense. Its only purpose is academic and to understand the text generation using RNNs.
    - Enhance code comprehension and documentation skills by commenting on provided starter code.
    
<br>

- Instructions:
    - Code Understanding: Begin by thoroughly reading and understanding the code. Comment each section/block of the provided code to demonstrate your understanding. For this, you are encouraged to add cells with experiments to improve your understanding

    - Model Overview: The starter code includes an LSTM model setup for sequence data processing. Familiarize yourself with the model architecture and its components. Once you are familiar with the provided model, feel free to change the model to experiment.

    - Training Function: Implement a function to train the LSTM model on the WikiText-2 dataset. This function should feed the training data into the model and perform backpropagation.

    - Text Generation Function: Create a function that accepts starting text (seed text) and a specified total number of words to generate. The function should use the trained model to generate a continuation of the input text.

    - Code Commenting: Ensure that all the provided starter code is well-commented. Explain the purpose and functionality of each section, indicating your understanding.

    - Submission: Submit your Jupyter Notebook with all sections completed and commented. Include a markdown cell with the full names of all contributing team members at the beginning of the notebook.
    
<br>

- Evaluation Criteria:
    - Code Commenting (60%): The clarity, accuracy, and thoroughness of comments explaining the provided code. You are suggested to use markdown cells for your explanations.

    - Training Function Implementation (20%): The correct implementation of the training function, which should effectively train the model.

    - Text Generation Functionality (10%): A working function is provided in comments. You are free to use it as long as you make sure to uderstand it, you may as well improve it as you see fit. The minimum expected is to provide comments for the given function.

    - Conclusions (10%): Provide some final remarks specifying the differences you notice between this model and the one used  for classification tasks. Also comment on changes you made to the model, hyperparameters, and any other information you consider relevant. Also, please provide 3 examples of generated texts.



In [1]:
!pip install portalocker>=2.0.0
!pip install torchtext

In [16]:
#PyTorch libraries
import torch
import torchtext
from torchtext.datasets import WikiText2
# Dataloader library
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
# neural layers
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from tqdm import tqdm

import numpy as np
import random

In [18]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    try:
        device = torch.device('mps')
    except:
        device = torch.device('cpu')
print(device)

cuda


In [19]:
train_dataset, val_dataset, test_dataset = WikiText2()

In [5]:
tokeniser = get_tokenizer('basic_english')
def yield_tokens(data):
    for text in data:
        yield tokeniser(text)

In [6]:
# Build the vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
#set unknown token at position 0
vocab.set_default_index(vocab["<unk>"])

In [7]:
seq_length = 50
def data_process(raw_text_iter, seq_length = 50):
    data = [torch.tensor(vocab(tokeniser(item)), dtype=torch.long) for item in raw_text_iter]
    data = torch.cat(tuple(filter(lambda t: t.numel() > 0, data))) #remove empty tensors
#     target_data = torch.cat(d)
    return (data[:-(data.size(0)%seq_length)].view(-1, seq_length),
            data[1:-(data.size(0)%seq_length-1)].view(-1, seq_length))

# # Create tensors for the training set
x_train, y_train = data_process(train_dataset, seq_length)
x_val, y_val = data_process(val_dataset, seq_length)
x_test, y_test = data_process(test_dataset, seq_length)

In [8]:
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

In [9]:
batch_size = 64  # choose a batch size that fits your computation resources
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [10]:
# Define the LSTM model
# Feel free to experiment
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, text, hidden):
        embeddings = self.embeddings(text)
        output, hidden = self.lstm(embeddings, hidden)
        decoded = self.fc(output)
        return decoded, hidden

    def init_hidden(self, batch_size):

        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))



vocab_size = len(vocab) # vocabulary size
emb_size = 100 # embedding size
neurons = 128 # the dimension of the feedforward network model, i.e. # of neurons
num_layers = 1 # the number of nn.LSTM layers
model = LSTMModel(vocab_size, emb_size, neurons, num_layers)


In [25]:
def train(model, epochs, optimizer, criterion):
    model = model.to(device=device)
    model.train()

    for epoch in range(epochs):
        total_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for i, (data, targets) in enumerate(train_loader):
            optimizer.zero_grad()

            # Move data to the device
            data, targets = data.to(device), targets.to(device)

            # Initialize hidden state
            hidden = model.init_hidden(data.size(0))

            # Forward pass
            outputs, hidden = model(data, hidden)

            # Calculate the loss
            loss = criterion(outputs.view(-1, len(vocab)), targets.view(-1))
            total_loss += loss.item()

            # Compute accuracy
            _, predicted = torch.max(outputs, 2)
            correct_predictions += (predicted == targets).sum().item()
            total_samples += targets.numel()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        loss = total_loss / len(train_loader)
        accuracy = correct_predictions / total_samples
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss:.4f}, Accuracy: {accuracy * 100:.2f}%')


In [57]:
# Call the train function
loss_function = nn.CrossEntropyLoss()
lr = 0.001
epochs = 10
optimiser = optim.Adam(model.parameters(), lr=lr)
train(model, epochs, optimiser,loss_function)

Epoch [1/10], Loss: 3.3948, Accuracy: 37.21%
Epoch [2/10], Loss: 3.3670, Accuracy: 37.66%
Epoch [3/10], Loss: 3.3538, Accuracy: 37.88%
Epoch [4/10], Loss: 3.3420, Accuracy: 38.06%
Epoch [5/10], Loss: 3.3307, Accuracy: 38.24%
Epoch [6/10], Loss: 3.3203, Accuracy: 38.40%
Epoch [7/10], Loss: 3.3106, Accuracy: 38.54%
Epoch [8/10], Loss: 3.3012, Accuracy: 38.68%
Epoch [9/10], Loss: 3.2919, Accuracy: 38.82%
Epoch [10/10], Loss: 3.2831, Accuracy: 38.94%


In [61]:
def generate_text(model, start_text, num_words, temperature=1.0):
    '''
    Generate text using the trained model.

    Parameters:
        model (LSTMModel): The trained LSTM model.
        start_text (str): The starting text or seed.
        num_words (int): The total number of words to generate.
        temperature (float): Controls the randomness of the generated text.

    Returns:
        str: The generated text.
    '''
    model.eval()

    words = tokeniser(start_text)
    hidden = model.init_hidden(1)

    with torch.no_grad():  # No need to track gradients during generation
        for _ in range(num_words):
            x = torch.tensor([[vocab[word] for word in words[-seq_length:]]], dtype=torch.long, device=device)
            y_pred, hidden = model(x, hidden)
            last_word_logits = y_pred[0][-1]
            p = F.softmax(last_word_logits / temperature, dim=0).cpu().numpy()
            word_index = np.random.choice(len(last_word_logits), p=p)
            words.append(vocab.lookup_token(word_index))

    return ' '.join(words)

In [63]:
# Generate some text
print(generate_text(model, start_text="I like", num_words=100))

i like a lot about the music we show gotten verses laugh and working on mainstream , right to the contrary . by an appearance of his time praised traits of differences , the characters are , this understanding romantic fundamentalist shaolin fictional <unk> . while staying in full producers worldwide , the book continued to townsend ' s promotional websites for hollywood about eight years . he is built by examining the second producer , highly varied behind all @-@ dose film , and promote <unk> of tv @-@ general today . richard <unk> of <unk> played <unk> mary as <unk>
